In [33]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py

In [34]:
with open('data/joy_division_lyrics.txt') as file:
    data = file.read().lower().split('\n')


In [35]:
chars_to_remove = "?()!-,.\"\'"
cleaned = []
for line in data:
    for del_char in chars_to_remove:
        line = line.replace(del_char, "")
    cleaned.append(line)
        

In [36]:
chars = sorted(list(set("\n".join(cleaned))))
print(chars)

['\n', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [37]:
sequence_length = 60
n_vocab = len(chars)
char_to_int = {char:num for num, char in enumerate(chars)}
lyrics = "\n".join(cleaned)

In [41]:
# make the input sequences and corresponding outputs!
input = []
output = []
for i in range(0, len(lyrics) - sequence_length):
    seq_in = lyrics[i:i+sequence_length]
    seq_out = lyrics[i+sequence_length]
    input.append([char_to_int[char] for char in seq_in])
    output.append(char_to_int[seq_out])
num_patterns = len(input)
input = np.reshape(input, (num_patterns, sequence_length,1))
# normalize
input = input / float(n_vocab)
output = to_categorical(output)

In [42]:
print(input.shape)

(18967, 60, 1)


In [43]:
model = Sequential()
model.add(LSTM(256, input_shape=(input.shape[1], input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [44]:
filepath = "weight_checkpoints/lyrics/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0,save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [45]:
model.fit(input, output, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-09-11 17:59:18.625050: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-11 17:59:18.625233: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


297/297 [==============================] - 46s 153ms/step - loss: 3.1211
Epoch 2/50
297/297 [==============================] - 46s 155ms/step - loss: 2.8938
Epoch 3/50
297/297 [==============================] - 46s 153ms/step - loss: 2.7552
Epoch 4/50
297/297 [==============================] - 46s 156ms/step - loss: 2.6624
Epoch 5/50
297/297 [==============================] - 46s 155ms/step - loss: 2.5766
Epoch 6/50
297/297 [==============================] - 46s 156ms/step - loss: 2.5469
Epoch 7/50
297/297 [==============================] - 47s 157ms/step - loss: 2.3967
Epoch 8/50
297/297 [==============================] - 47s 157ms/step - loss: 2.2627
Epoch 9/50
297/297 [==============================] - 46s 155ms/step - loss: 2.1814
Epoch 10/50
297/297 [==============================] - 46s 153ms/step - loss: 2.0113
Epoch 11/50
211/297 [====================>.........] - ETA: 13s - loss: 1.8715

KeyboardInterrupt: 

In [55]:
model = Sequential()
model.add(LSTM(256, input_shape=(input.shape[1], input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.load_weights("weight_checkpoints/lyrics/weights-improvement-10-2.0196-bigger.hdf5")

In [56]:
start = np.random.randint(0, len(input) - 1)
int_to_char = {num:char for num, char in enumerate(chars)}
pattern = input[start]
pred_output = []
for note_index in range(500):
    # make it 1,50,1 instead of 50,1
    pred_input = np.reshape(pattern, (1,len(pattern), 1))
    # normalize input
    pred_input = pred_input / float(n_vocab)
    # predict w/ model
    pred = model.predict(pred_input, verbose=0)
    # get the most likely index of the next note
    index = np.argmax(pred)
    # convert to note form 
    result = int_to_char[index]
    # append the result to the output
    pred_output.append(result)
    pattern = np.append(pattern,index)
    pattern = pattern[1:len(pattern)]
    

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [57]:
"".join(pred_output)

'loree the sie seak the cee age\nlo lising in the ceeling ieeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feeling feel'